# Hypothetical Document Embeddings
This notebook goes over how to use Hypothetical Document Embeddings (HyDE), as described in [this paper](http://boston.lti.cs.cmu.edu/luyug/HyDE/HyDE.pdf). 

At a high level, HyDE is an embedding technique that takes queries, generates a hypothetical answer, and then embeds that generated document and uses that as the final example. 

In order to use HyDE, we therefor need to provide a base embedding model, as well as an LLMChain that can be used to generate those documents. By default, the HyDE class comes with some default prompts to use (see the paper for more details on them), but we can also create our own.

In [6]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings, HypotheticalDocumentEmbedder
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
base_embeddings = OpenAIEmbeddings()
llm = OpenAI()

In [3]:
# Load with `web_search` prompt
embeddings = HypotheticalDocumentEmbedder.from_llm(llm, base_embeddings, "web_search")

In [4]:
# Now we can use it as any embedding class!
embeddings.embed_query("Where is the Taj Mahal?")

[-0.0056639546528458595,
 0.0012234329478815198,
 -0.010081776417791843,
 0.006049065850675106,
 -0.0051316870376467705,
 0.02350744605064392,
 -0.02825402282178402,
 -0.012035512365400791,
 -0.005247533321380615,
 -0.008196922950446606,
 0.015391930006444454,
 -0.001557665760628879,
 0.004386512096971273,
 0.013225287199020386,
 -0.018685726448893547,
 0.0037352668587118387,
 0.025185655802488327,
 -0.01820981688797474,
 -0.00990644097328186,
 -0.023457350209355354,
 -0.012217109091579914,
 -0.001012091408483684,
 -0.005166127812117338,
 0.018410200253129005,
 -0.01640636846423149,
 0.008766761980950832,
 0.017721382901072502,
 -0.03832327574491501,
 0.00513481767848134,
 -0.006340247578918934,
 -0.02174157090485096,
 -0.020226173102855682,
 -0.028604691848158836,
 -0.03296302631497383,
 -0.018986301496624947,
 0.01688227988779545,
 -0.0032844047527760267,
 -0.006431046407669783,
 0.013325478881597519,
 0.006656477227807045,
 0.029356129467487335,
 0.0122108468785882,
 0.0065562855452

## Using our own prompts
Besides using preconfigured prompts, we can also easily construct our own prompts and use those in the LLMChain that is generating the documents. This can be useful if we know the domain our queries will be in, as we can condition the prompt to generate text more similar to that.

In the example below, let's condition it generate text about a state of the union address (because we will use that in the next example).

In [8]:
prompt_template = """Please answer the user's question about the most recent state of the union address
Question: {question}
Answer:"""
prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=base_embeddings)

In [10]:
embeddings.embed_query("Where is the Taj Mahal?")

[-0.007121483329683542,
 -0.008868738077580929,
 -0.02119413949549198,
 0.0008152805385179818,
 0.009272435680031776,
 0.01757347397506237,
 -0.035096485167741776,
 -0.01201632060110569,
 -0.0032831993885338306,
 -0.013422954827547073,
 0.009814905002713203,
 -0.004973684437572956,
 0.019251342862844467,
 -0.019125187769532204,
 -0.014066348783671856,
 -0.005796849261969328,
 0.015744218602776527,
 -0.019541501998901367,
 0.010287988930940628,
 -0.012041551060974598,
 -0.013410339131951332,
 -0.00039640453178435564,
 -0.0032059289515018463,
 0.013094950467348099,
 -0.02926809713244438,
 0.014078964479267597,
 0.014545739628374577,
 -0.043347060680389404,
 7.761525921523571e-05,
 -0.0017756399465724826,
 -0.031185662373900414,
 -0.023717250674962997,
 -0.032724760472774506,
 -0.044406767934560776,
 -0.010798919014632702,
 -0.004532139748334885,
 -0.019882122054696083,
 -0.010906150564551353,
 0.01718239113688469,
 0.013650035485625267,
 0.03247245028614998,
 0.003187005640938878,
 -0.00

## Using HyDE
Now that we have HyDE, we can use it as we would any other embedding class! Here is using it to find similar passages in the state of the union example.

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [16]:
docsearch = FAISS.from_texts(texts, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [19]:
print(docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 
